# CFB Pick 'em
Description: Get CFB lines and export to a spreadsheet.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import math
from datetime import date

In [2]:
def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

### Get Data

In [3]:
# CFB lines url
espn_url = 'https://www.espn.com/college-football/lines'

In [4]:
# get raw data
page = requests.get(espn_url)
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="fittPageContainer") # found by inspecting html

### Create Table

In [5]:
output = [i.text for i in results.find_all("td", class_="Table__TD")]
output[0:7]

['Arkansas Razorbacks',
 '4-0 (4-0-0)',
 '48.5',
 '+600',
 '11.0%',
 'Georgia Bulldogs',
 '4-0 (3-1-0)']

In [6]:
# 5 entries for each team
rows = int(len(output)/5)
table = pd.DataFrame(np.reshape(output, (rows, 5)), columns = ['TEAM', 'RECORD', 'RAW_LINE', 'ML', 'FPI'])
table.head()

,TEAM,RECORD,RAW_LINE,ML,FPI
0,Arkansas Razorbacks,4-0 (4-0-0),48.5,+600,11.0%
1,Georgia Bulldogs,4-0 (3-1-0),-16.5,-900,89.0%
2,Michigan Wolverines,4-0 (3-1-0),43.5,+110,68.7%
3,Wisconsin Badgers,1-2 (1-2-0),-2.0,-130,31.3%
4,Tennessee Volunteers,2-2 (1-3-0),65.5,+120,47.3%


In [7]:
# Change column types
table = table.astype({"TEAM": str, "RECORD": str, "RAW_LINE": float, "ML": str, "FPI":str})

In [8]:
# Add in an indicators for the game number (used to join later)
game_id = []
for i in range(rows):
    game_id.append(math.floor(i/2))
table.insert(0, 'GAME_ID', game_id)
table.head()

,GAME_ID,TEAM,RECORD,RAW_LINE,ML,FPI
0,0,Arkansas Razorbacks,4-0 (4-0-0),48.5,+600,11.0%
1,0,Georgia Bulldogs,4-0 (3-1-0),-16.5,-900,89.0%
2,1,Michigan Wolverines,4-0 (3-1-0),43.5,+110,68.7%
3,1,Wisconsin Badgers,1-2 (1-2-0),-2.0,-130,31.3%
4,2,Tennessee Volunteers,2-2 (1-3-0),65.5,+120,47.3%


In [9]:
# create favs and dogs tables to get the lines and o/u seperated
favs = table[table['RAW_LINE'] < 0]
dogs = table[table['RAW_LINE'] > 0]

# favs have the Raw Line as the line - this is fine, we just need to rename
favs = favs.rename(columns={"RAW_LINE": "LINE"})

# for the dogs, rename column for the o/u
dogs = dogs.rename(columns={'RAW_LINE':'O/U'})

# create df with just the data to merge on (game_id and o/u or line)
dogs_to_join = dogs.copy()[['GAME_ID', 'O/U']]
favs_to_join = favs.copy()[['GAME_ID', 'LINE']]

# merge
dogs = dogs.merge(favs_to_join)
dogs['LINE'] = dogs['LINE'] * -1
favs = favs.merge(dogs_to_join)

# concat
full = pd.concat([dogs, favs],sort=True)[['GAME_ID','TEAM','RECORD','O/U','LINE']]
full = full.sort_values('GAME_ID')

In [10]:
# Add in School Reference
school_lookup = pd.read_csv("lookup-school-abbrv.csv")
school_lookup['TEAM'] = school_lookup['SCHOOL'] + ' ' + school_lookup['NICKNAME']
school_lookup.drop(['SCHOOL', 'NICKNAME'], axis=1,inplace=True)

# merge abbreviations
full_abbrv = full.merge(school_lookup, how='left')
full_abbrv.head()

,GAME_ID,TEAM,RECORD,O/U,LINE,ABBRV
0,0,Arkansas Razorbacks,4-0 (4-0-0),48.5,16.5,ARK
1,0,Georgia Bulldogs,4-0 (3-1-0),48.5,-16.5,GEOR
2,1,Michigan Wolverines,4-0 (3-1-0),43.5,2.0,MICH
3,1,Wisconsin Badgers,1-2 (1-2-0),43.5,-2.0,WIS
4,2,Tennessee Volunteers,2-2 (1-3-0),65.5,2.5,TEN


In [11]:
full_abbrv[full_abbrv['ABBRV'].isna()]

,GAME_ID,TEAM,RECORD,O/U,LINE,ABBRV


In [12]:
# save out as dataset
todays_date = str(date.today())
full_abbrv.to_csv('data/'+todays_date+'-cfb-lines.csv', index=False)

In [13]:
full_abbrv.head(10)

,GAME_ID,TEAM,RECORD,O/U,LINE,ABBRV
0,0,Arkansas Razorbacks,4-0 (4-0-0),48.5,16.5,ARK
1,0,Georgia Bulldogs,4-0 (3-1-0),48.5,-16.5,GEOR
2,1,Michigan Wolverines,4-0 (3-1-0),43.5,2.0,MICH
3,1,Wisconsin Badgers,1-2 (1-2-0),43.5,-2.0,WIS
4,2,Tennessee Volunteers,2-2 (1-3-0),65.5,2.5,TEN
5,2,Missouri Tigers,2-2 (0-4-0),65.5,-2.5,MIZZ
6,3,Charlotte 49ers,3-1 (3-1-0),54.0,10.5,CLT
7,3,Illinois Fighting Illini,1-4 (3-2-0),54.0,-10.5,ILL
8,4,Georgia Tech Yellow Jackets,2-2 (2-1-0),58.0,3.0,GT
9,4,Pittsburgh Panthers,3-1 (3-1-0),58.0,-3.0,PITT
